# **Set according to environment (e.g. local, Google Colab...)**

In [1]:
project_folder = ''

# **Body**

In [2]:
from custom_libraries.image_dataset import *
from custom_libraries.miscellaneous import *
import numpy as np

In [3]:
def create_model(num_units):
    model = tf.keras.Sequential()
    model.add(
        tf.keras.layers.Dense(units=num_units, activation=None, kernel_initializer=tf.keras.initializers.HeNormal))
    model.add(tf.keras.layers.LeakyReLU(alpha=.01))
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE),
                  metrics=['binary_crossentropy', 'acc'])

    return model

In [4]:
bs = 256
trials = 10
epochs = 2000
trees_set = [1, 32]

classes = np.load(project_folder + 'results/classes.npy', allow_pickle=True)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=60, mode='min')

history = np.zeros((len(classes), trials, len(trees_set), 2))
#history = np.load(project_folder+'results/fcnn_history.npy', allow_pickle=True)

for j, (t1, t2, ds) in enumerate(classes):

    print(f"Dataset: {ds} / Pair: {t1}-{t2}")

    if history[j, 0, 0, 0] != 0:
        continue

    test_ds = ImageDataset(ds, 'test', data_dir=None, USPS_dir=project_folder + 'USPS/')
    train_ds = ImageDataset(ds, 'train', data_dir=None, USPS_dir=project_folder + 'USPS/')

    for x in [train_ds, test_ds]:
        x.filter(t1, t2, overwrite=True)
        x.shuffle()
        x.normalize()
        if x.images.shape[1:3] == (28, 28):
            x.pad()
        x.vectorize(True)

    X_train, y_train, X_valid, y_valid = train_ds.subset(shard=True, shard_number=trials, validation=True,
                                                         validation_size=len(test_ds.images))
    test_set = tf.data.Dataset.from_tensor_slices((test_ds.images, test_ds.labels)).batch(bs)

    for k, trees in enumerate(trees_set):

        print(f"{trees}-FCNN")

        for i in range(trials):

            if history[j, i, k, 0] != 0:
                continue

            print(f"Trial {i + 1}")

            model = create_model(num_units=2 * trees)

            train_set = tf.data.Dataset.from_tensor_slices((X_train[i], y_train[i])).batch(bs)
            valid_set = tf.data.Dataset.from_tensor_slices((X_valid[i], y_valid[i])).batch(bs)

            with tf.device('/device:GPU:0'):

                fit_history = model.fit(x=train_set, batch_size=bs, epochs=epochs,
                                        validation_data=valid_set, validation_batch_size=bs,
                                        callbacks=[callback], verbose=0)
                print_fit_history(fit_history, epochs)

                evaluate_history = model.evaluate(x=test_set, batch_size=bs, verbose=0)
                print_evaluate_history(evaluate_history)

                history[j, i, k] = evaluate_history[1:]

                np.save(project_folder + 'results/fcnn_history.npy', history,
                        allow_pickle=True)

Dataset: mnist / Pair: 3-5
1-FCNN
Trial 1
Epochs: 257/2000 - Train BCE: 0.0498, accuracy: 99.48% - Validation BCE: 0.127, accuracy: 94.76%
Test BCE: 0.1512, accuracy: 94.64%
Trial 2
Epochs: 960/2000 - Train BCE: 0.0289, accuracy: 99.69% - Validation BCE: 0.1236, accuracy: 97.38%
Test BCE: 0.1917, accuracy: 95.37%
Trial 3
Epochs: 146/2000 - Train BCE: 0.0289, accuracy: 99.79% - Validation BCE: 0.1508, accuracy: 95.79%
Test BCE: 0.1266, accuracy: 95.74%
Trial 4
Epochs: 197/2000 - Train BCE: 0.0379, accuracy: 99.48% - Validation BCE: 0.0992, accuracy: 95.79%
Test BCE: 0.1252, accuracy: 95.06%
Trial 5
Epochs: 202/2000 - Train BCE: 0.046, accuracy: 98.76% - Validation BCE: 0.1363, accuracy: 95.26%
Test BCE: 0.1086, accuracy: 95.85%
Trial 6
Epochs: 143/2000 - Train BCE: 0.0697, accuracy: 98.34% - Validation BCE: 0.1597, accuracy: 96.32%
Test BCE: 0.1225, accuracy: 94.9%
Trial 7
Epochs: 788/2000 - Train BCE: 0.0433, accuracy: 99.79% - Validation BCE: 0.2219, accuracy: 95.79%
Test BCE: 0.1725,

In [5]:
history = np.load(project_folder + 'results/fcnn_history.npy', allow_pickle=True)
print("RESULTS:")
for j, (t1, t2, ds) in enumerate(classes):
    print(f"Dataset: {ds} / Pair: {t1}-{t2}")
    for k, trees in enumerate(trees_set):
        print(f"{trees}-FCNN")
        print(f"Accuracy: mean = {round(np.mean(history[j,:,k,1]), 4)}, standard deviation = {round(np.std(history[j,:,k,1]), 4)}")

RESULTS:
Dataset: mnist / Pair: 3-5
0-FCNN
Accuracy: mean = 0.9539, standard deviation = 0.004
1-FCNN
Accuracy: mean = 0.9674, standard deviation = 0.0042
Dataset: fmnist / Pair: 0-6
0-FCNN
Accuracy: mean = 0.8314, standard deviation = 0.0062
1-FCNN
Accuracy: mean = 0.832, standard deviation = 0.006
Dataset: kmnist / Pair: 2-6
0-FCNN
Accuracy: mean = 0.8697, standard deviation = 0.0107
1-FCNN
Accuracy: mean = 0.9064, standard deviation = 0.0121
Dataset: emnist / Pair: 14-17
0-FCNN
Accuracy: mean = 0.9808, standard deviation = 0.0047
1-FCNN
Accuracy: mean = 0.9851, standard deviation = 0.0021
Dataset: svhn / Pair: 5-6
0-FCNN
Accuracy: mean = 0.5615, standard deviation = 0.0683
1-FCNN
Accuracy: mean = 0.6744, standard deviation = 0.1192
Dataset: usps / Pair: 3-5
0-FCNN
Accuracy: mean = 0.9028, standard deviation = 0.0216
1-FCNN
Accuracy: mean = 0.912, standard deviation = 0.0116
Dataset: cifar10 / Pair: 3-5
0-FCNN
Accuracy: mean = 0.5427, standard deviation = 0.0271
1-FCNN
Accuracy: mean